In [1]:
import os
import skimage.io as io
import skimage.transform as trans
import random
from keras import metrics
from keras import backend as K
from keras.initializers import *
from keras.layers import *
from keras.models import *
from keras.models import load_model
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.metrics import binary_crossentropy as bc_met
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

Using TensorFlow backend.


In [2]:
random.seed('beachclassification')
model_initializer = glorot_uniform
kernel_init = model_initializer(seed=random.randint(0,5))
model_learning_rate = 0.00001

In [3]:
def sorensen_dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return coef

def sorensen_dice_coef_loss(y_true, y_pred):
    return 1 - sorensen_dice_coef(y_true, y_pred)

In [ ]:
def generateUnet():
    inputs = Input(shape=(256,256,4))
    conv1 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(inputs)
    conv1 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool1)
    conv2 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool2)
    conv3 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    conv4 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool3)
    conv4 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv4)
    drop4 = Dropout(0.01)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(drop4)
    
    conv5 = Conv2D(1024, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool4)
    conv5 = Conv2D(1024, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv5)
    drop5 = Dropout(0.01)(conv5)
    
    up6 = concatenate([Conv2DTranspose(512, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(drop5), drop4], axis=3)
    conv6 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up6)
    conv6 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv6)
    
    up7 = concatenate([Conv2DTranspose(256, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(conv6), conv3], axis=3)
    conv7 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up7)
    conv7 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv7)
    
    up8 = concatenate([Conv2DTranspose(128, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(conv7), conv2], axis=3)
    conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up8)
    conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv8)
    
    up9 = concatenate([Conv2DTranspose(64, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(conv8), conv1], axis=3)
    conv9 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up9)
    conv9 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv9)
    conv9 = Conv2D(2, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv9)
    
    conv10 = Conv2D(1, (1,1), activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    model.compile(optimizer=Adam(lr=model_learning_rate), loss=binary_crossentropy, metrics=[bc_met])
    
    return model

In [5]:
def generateRealUnet():
    inputs = Input(shape=(256,256,4))
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs=inputs,outputs=conv10)

    model.compile(optimizer = Adam(lr = model_learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [6]:
model = generateRealUnet()

In [2]:
K.image_data_format()

'channels_last'

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 2368        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [ ]:
#class ImageData():
    
    #def __init__(self, image_dir, label_dir):
        #self.image = []
        #self.label = []
        #for file in os.listdir(image_dir):
            #if file.endswith(".tif"):
                #self.image.append(io.imread(os.path.join(image_dir, file)))
        #for file in os.listdir(label_dir):
        #    if file.endswith(".tif"):
        #        self.label.append(io.imread(os.path.join(label_dir, file)))
    
    #def getAllImages(self):
        #return np.array(self.image)
    
    #def getAllLabels(self):
        #return np.array(self.label)

In [3]:
def getImageryInRange(range_of_ids):
    images = []
    for val in range(range_of_ids[0], range_of_ids[1]):
        path = "/home/paperspace/ciaran/data/train_normalised/blue_anchor_" + str(val) + ".tif"
        images.append(io.imread(path))
    return np.array(images)
def getLabelsInRange(range_of_ids):
    labels = []
    for val in range(range_of_ids[0], range_of_ids[1]):
        path = "/home/paperspace/ciaran/data/label/blue_anchor_" + str(val) + ".tif"
        labels.append(np.expand_dims(io.imread(path), axis=2))
    return np.array(labels)

In [4]:
training_images = getImageryInRange((0, 3000))
training_labels = getLabelsInRange((0, 3000))

In [ ]:
#images = ImageData("/home/paperspace/ciaran/data/train_normalised/",
#                   "/home/paperspace/ciaran/data/label/")
#all_images = images.getAllImages()
#training_images = all_images[((40 * len(all_images))//100):((60 * len(all_images))//100)]
#all_labels = images.getAllLabels()
#training_labels = all_labels[((40 * len(all_labels))//100):((60 * len(all_labels))//100)]

In [5]:
import h5py
import re
from importlib import reload
from keras.callbacks import EarlyStopping, TensorBoard
from keras.models import load_model
from keras.models import save_model

In [15]:
print(training_images[0].shape)
print(training_labels[0].shape)

(256, 256, 4)
(256, 256, 1)


In [ ]:
t_board = TensorBoard(log_dir="./models/1e-5/logs", histogram_freq=0, write_grads=False, write_graph=True, write_images=False)
stop_early = EarlyStopping(patience=30)
model.fit(training_images, training_labels, validation_split=0.2, epochs=30, batch_size=16, shuffle=True, callbacks=[stop_early, t_board])
model.save("./models/1e-5/model")

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 174s 72ms/step - loss: 0.6537 - acc: 0.9817 - val_loss: 0.6556 - val_acc: 0.9320
Epoch 2/30
2400/2400 [==============================] - 168s 70ms/step - loss: 0.6504 - acc: 0.9842 - val_loss: 0.6547 - val_acc: 0.9320
Epoch 3/30
2400/2400 [==============================] - 168s 70ms/step - loss: 0.6495 - acc: 0.9842 - val_loss: 0.6539 - val_acc: 0.9320
Epoch 4/30
2400/2400 [==============================] - 168s 70ms/step - loss: 0.6487 - acc: 0.9842 - val_loss: 0.6532 - val_acc: 0.9320
Epoch 5/30
2400/2400 [==============================] - 168s 70ms/step - loss: 0.6478 - acc: 0.9842 - val_loss: 0.6525 - val_acc: 0.9320
Epoch 6/30
2400/2400 [==============================] - 168s 70ms/step - loss: 0.6470 - acc: 0.9842 - val_loss: 0.6518 - val_acc: 0.9320
Epoch 7/30
2384/2400 [============================>.] - ETA: 1s - loss: 0.6462 - acc: 0.9841

In [6]:
model = load_model('./models/1e-5/model')